In [1]:
import pandas as pd
import soundfile as sf
from tqdm import tqdm
import ast
tqdm.pandas()

In [2]:
df = pd.read_csv('./inputs/train_audios/train_metadata.csv')
additional = pd.read_csv('./inputs/train_audios/BirdClef2024_additional.csv')
duplicate = pd.read_csv('./inputs/train_audios/duplicates.csv')
master_df = pd.read_csv('./inputs/train_audios/eBird_Taxonomy_v2021.csv')

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df['version'] = 2024
df['filepath'] = 'train_audio/' + df['filename']

In [4]:
additional = additional.rename(columns={'lat': 'latitude', 'lng': 'longitude'})
additional['version'] = 2024
additional['rating'] = additional['q'].apply(lambda x: 5 if x == 'A' else 4 if x == 'B' else 3 if x == 'C' else 2 if x == 'D' else 1)
spc2sci = master_df[["SPECIES_CODE","SCI_NAME"]].set_index("SCI_NAME")["SPECIES_CODE"].to_dict()
additional['scientific_name'] = additional['gen'] + ' ' + additional['sp']
additional["secondary_labels"] = additional.also.apply(eval).apply(lambda x: [spc2sci[t] for t in x if t in spc2sci.keys()])

In [5]:
def get_duration(row, audio_root):
    audio_path = audio_root + row['filename']
    error_files = []
    try:
        info = sf.info(audio_path)
        return info.duration
    except:
        error_files.append(row['filename'])
        return None

In [6]:
additional['filename'] = additional['primary_label'] + '/' + additional['file'] + '.mp3'
additional['filepath'] = 'additional_audio/' + additional['filename']

In [7]:
audio_root = './inputs/train_audios/train_audio/'
df['duration'] = df.progress_apply(lambda x: get_duration(x, audio_root), axis=1)
additional_audio_root = './inputs/train_audios/additional_audio/'
additional['duration'] = additional.progress_apply(lambda x: get_duration(x, additional_audio_root), axis=1)

100%|██████████| 24279/24279 [01:03<00:00, 379.36it/s]


In [8]:
additional['duration']

0         57.696000
1         61.272000
2         23.249000
3         13.009000
4         44.747781
            ...    
24274    135.768000
24275     28.056000
24276     28.080000
24277     47.616000
24278     18.240000
Name: duration, Length: 24279, dtype: float64

In [9]:
dup_files = set(list(duplicate['filename_a']) + list(duplicate['filename_b']))
df = df[~df['filename'].isin(dup_files)]

In [10]:
common_columns = df.columns.intersection(additional.columns)
total = pd.concat([df[common_columns], additional[common_columns]], ignore_index=True)

In [11]:
def safe_sample(group, n_samples):
    if len(group) > n_samples:
        high_rating_group = group[group['rating'] >= 4]
        n = min(len(high_rating_group), n_samples)
        return high_rating_group.sample(n=n, random_state=1)
    else:
        return group.sample(n=len(group), random_state=1)

sampling_total = total.groupby('primary_label').apply(safe_sample, n_samples=500).reset_index(drop=True)

In [12]:
df.to_csv('./inputs/train_audios/train_metadata_v2.csv', index=False)
sampling_total.to_csv('./inputs/train_audios/train.csv', index=False)